In [9]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from src.config import get_parser
from src.util import make_faces
from pytorch3d.structures import Meshes

config = get_parser().parse_args(args=[])

blueprint = np.load(os.path.join(config.data_dir, config.blueprint)) 
points = torch.tensor(blueprint['points'])
normals = torch.tensor(blueprint['normals'])
points = F.interpolate(points, size=config.data_blueprint_size,
                               mode='bicubic', align_corners=True)
normals = F.interpolate(normals, size=config.data_blueprint_size, 
                        mode='bicubic', align_corners=True)   
points.shape, normals.shape

(torch.Size([1, 3, 640, 640]), torch.Size([1, 3, 640, 640]))

In [4]:
faces = torch.tensor(make_faces(config.data_blueprint_size, config.data_blueprint_size))
faces.shape

torch.Size([816642, 3])

In [7]:
bs = 5
verts = points.reshape(1, 3, -1).permute(0, 2, 1).expand(bs, -1, -1)
face_id = faces[None].expand(bs, -1, -1)
verts.shape, face_id.shape

(torch.Size([5, 409600, 3]), torch.Size([5, 816642, 3]))

In [10]:
trg_mesh = Meshes(verts=verts, faces=face_id)
trg_mesh

In [13]:
from src.loss.edge_loss import EdgeLoss

config.data_patch_size = config.data_blueprint_size
edgeLoss = EdgeLoss(config)
edgeLoss(verts)

tensor(8.8474e-05)

In [ ]:
def _compute_laplacian_packed(self, refresh: bool = False):
    """
    Computes the laplacian in packed form.
    The definition of the laplacian is
    L[i, j] =    -1       , if i == j
    L[i, j] = 1 / deg(i)  , if (i, j) is an edge
    L[i, j] =    0        , otherwise
    where deg(i) is the degree of the i-th vertex in the graph

    Returns:
        Sparse FloatTensor of shape (V, V) where V = sum(V_n)

    """   
    verts_packed = self.verts_packed()  # (sum(V_n), 3)
    edges_packed = self.edges_packed()  # (sum(E_n), 3)
    V = verts_packed.shape[0]  # sum(V_n)

    e0, e1 = edges_packed.unbind(1)

    idx01 = torch.stack([e0, e1], dim=1)  # (sum(E_n), 2)
    idx10 = torch.stack([e1, e0], dim=1)  # (sum(E_n), 2)
    idx = torch.cat([idx01, idx10], dim=0).t()  # (2, 2*sum(E_n))

    # First, we construct the adjacency matrix,
    # i.e. A[i, j] = 1 if (i,j) is an edge, or
    # A[e0, e1] = 1 &  A[e1, e0] = 1
    ones = torch.ones(idx.shape[1], dtype=torch.float32, device=self.device)
    A = torch.sparse.FloatTensor(idx, ones, (V, V))

    # the sum of i-th row of A gives the degree of the i-th vertex
    deg = torch.sparse.sum(A, dim=1).to_dense()

    # We construct the Laplacian matrix by adding the non diagonal values
    # i.e. L[i, j] = 1 ./ deg(i) if (i, j) is an edge
    deg0 = deg[e0]
    deg0 = torch.where(deg0 > 0.0, 1.0 / deg0, deg0)
    deg1 = deg[e1]
    deg1 = torch.where(deg1 > 0.0, 1.0 / deg1, deg1)
    val = torch.cat([deg0, deg1])
    L = torch.sparse.FloatTensor(idx, val, (V, V))

    # Then we add the diagonal values L[i, i] = -1.
    idx = torch.arange(V, device=self.device)
    idx = torch.stack([idx, idx], dim=0)
    ones = torch.ones(idx.shape[1], dtype=torch.float32, device=self.device)
    L -= torch.sparse.FloatTensor(idx, ones, (V, V))

    self._laplacian_packed = L
    
self = meshes    